##### [모델 성능 개선 - 튜닝]
- scikit-learn에서는 튜닝을 위한 클래스 제공
    * GridSearchCV
    * RandomedSearchCV
    * CV 즉, 교차검증 함께 진행
    * 시간이 오래 걸림

[1] 모듈 로딩 및 데이터 준비 <HR>

In [171]:
# --------------------------------------------------
# [1-1] 모듈 로딩
# --------------------------------------------------
# 데이터 전처리
import pandas as pd
import numpy as np
import preprocessing

# 시각화
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

# 머신러닝 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV    # 튜닝 관련
from sklearn.neighbors import KNeighborsClassifier                      # 학습 알고리즘
from sklearn.model_selection import train_test_split                    # 데이터셋 관련
from sklearn.preprocessing import StandardScaler

In [172]:
# --------------------------------------------------
# [1-2] 데이터 준비
# --------------------------------------------------
FILE_NAME = '../Data/iris.csv'
irisDF = pd.read_csv(FILE_NAME)

In [173]:
# --------------------------------------------------
# [1-3] 데이터 확인
# --------------------------------------------------
# display(irisDF.head(3))
# display(irisDF.info())
preprocessing.check_data(irisDF)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


None

'\n'

[2] 데이터 전처리 <HR>
- 기본 데이터 전처리 : 결측치, 중복값, 피쳐별 분포, 이상치
- 학습관련 전처리 : 학습 알고리즘에 따른 처리, 피쳐와 타겟, 피쳐와 피쳐
- 학습관련 분리 : 피쳐와 타겟 분리, 학습용/검증용/테스트용 분리
- 학습데이터 전처리 : 이상치, 스케일러, 인코딩...
- 검증/테스트용 데이터 : 학습 알고리즘에 대입하기 위한 형태 맞춤 진행 
    - ★ 이상치 그대로/변경 여부 선택

In [174]:
# --------------------------------------------------
# [2-1] 피쳐와 타겟 분리
# --------------------------------------------------
featureDF = irisDF.drop('variety', axis=1)
targetSR = irisDF['variety']

# Train / Test 분리(검증은 CV로 처리)
train_feature, test_feature, train_target, test_target = train_test_split(
    featureDF,
    targetSR,
    test_size=0.2,
    random_state=42,
    stratify=targetSR
)

In [175]:
# --------------------------------------------------
# [2-2] 학습 알고리즘을 위한 전처리 : 거리기반 알고리즘 스케일러
# --------------------------------------------------
std_scaler = StandardScaler()

# Train에 대해서만 fit
train_feature_scaled = std_scaler.fit_transform(train_feature)

# Test는 transform만 , fit은 이미 완료
test_feature_scaled = std_scaler.transform(test_feature)

[3] 학습 및 검증, 하이퍼파라미터 찾기 <hr>

**- GridSearchCV: 모든 파라미터 조합으로 모델 생성 및 학습/검증 진행**

In [ ]:
# 1) 기본 모델
knn = KNeighborsClassifier()

# 2) 그리드 탐색용 하이퍼파라미터 설정
#    키 -> 학습 알고리즘의 매개변수 즉, 속성명
#    값 -> 학습 알고리즘의 매개변수 즉, 속성에 적용할 수 있는 값들
param_grid = {'n_neighbors':[1, 3, 5, 7, 9, 11, 13, 15],
              'weights' : ['uniform', 'distance'],
              'p':[1, 2]}    # 1: 맨해튼 거리, 2: 유클리드 거리

# 3) GridSearchCV 설정
grid_search = GridSearchCV(estimator=knn,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=1)

# 4) 학습 (★ 반드시 Train 데이터만 사용)
grid_search.fit(train_feature_scaled, train_target)

# => 학습 즉, fit() 이후 모델 파라미터(파라미터이름_)들
print("▶ GridSearchCV 최적 하이퍼파라미터: ", grid_search.best_params_)
print("▶ GridSearchCV CV 최고 정확도    : ", grid_search.best_score_)

print(grid_search.cv_results_.keys())
# resultDF = pd.DataFrame(grid_search.cv_results_)

# display("▶ 교차검증 결과    : ", resultDF)

# 5) 최적 모델로 Test 성능 평가
best_knn_grid = grid_search.best_estimator_
y_pred_grid = best_knn_grid.predict(test_feature_scaled)

print("\n▶ Test Accuracy (GridSearch 최적 모델): ", best_knn_grid.score(test_feature_scaled, test_target))

Fitting 5 folds for each of 32 candidates, totalling 160 fits
▶ GridSearchCV 최적 하이퍼파라미터:  {'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
▶ GridSearchCV CV 최고 정확도    :  0.9666666666666668
dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_neighbors', 'param_p', 'param_weights', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

▶ Test Accuracy (GridSearch 최적 모델):  0.9333333333333333


**- RandomizedSearchCV : GridSearchCV의 단점인 시간 개선 튜닝 방법**

In [177]:
# 랜덤 모듈
from scipy.stats import randint

# 1) 기본 모델
knn = KNeighborsClassifier()

# 2) 랟덤 탐색용 분포 설정
param_dist = {'n_neighbors':randint(1, 31), # 1 ~ 30 사이의 정수
              'weights' : ['uniform', 'distance'],
              'p':[1, 2]}    # 1: 맨해튼 거리, 2: 유클리드 거리

# 3) GridSearchCV 설정
random_search = RandomizedSearchCV(estimator=knn,
                                  param_distributions=param_dist,
                                  
                                  n_iter=20,   # 20개 조합만 랜덤으로 시도
                                  cv=5,
                                  scoring='accuracy',
                                  n_jobs=-1,
                                  random_state=42,
                                  verbose=2)

# 4) 학습 (★ 반드시 Train 데이터만 사용)
random_search.fit(train_feature_scaled, train_target)

print("▶ RandomizedSearchCV 최적 하이퍼파라미터: ", random_search.best_params_)
print("▶ RandomizedSearchCV CV 최고 정확도    : ", random_search.best_score_)

# 5) 최적 모델로 Test 성능 평가
best_knn_rand = random_search.best_estimator_

print("\n▶ Test Accuracy (RandomSearch 최적 모델): ", best_knn_rand.score(test_feature_scaled, test_target))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
▶ RandomizedSearchCV 최적 하이퍼파라미터:  {'n_neighbors': 24, 'p': 1, 'weights': 'distance'}
▶ RandomizedSearchCV CV 최고 정확도    :  0.975

▶ Test Accuracy (RandomSearch 최적 모델):  0.9333333333333333


#### [데이터 전처리 - 1 : 인코딩 & 디코딩]
- 범주형 => 수치형 : 인코딩(Encoding)
- 수치형 => 범주형 : 디코딩(Decoding)
- scikit-learn의 preprocessing 서브 패키지에 제공
    * LabelEncoder   : 타겟/라벨/클래스 컬럼 전용 => 0 ~ 클래스개수-1
    * OrdinalEncoder : 범주형 피쳐에 대한 인코딩 => 정수값 변환, 순서가 중요한 피쳐
        - 예: 만족도, 석차, 등급, ...
    * OneHotEncoder  : 범주형 피쳐에 대한 인코딩 => 값의 의미가 없고 데이터 의미가 중요한 피쳐
        - 예: 성별, 혈액형, 도시명, ...
    * TargetEncoder  : OneHotEncoder에 대한 대안. 타겟/라벨/클래스 컬럼의 데이터 타입에 따른 인코딩 진행
        - 예: 이진, 다중, 수치형, 연속형 ...


[1] LabelEncoder <hr>

In [178]:
# 관련 모듈 로딩
from sklearn.preprocessing import LabelEncoder

# 인스턴스 생성
lencoder = LabelEncoder()

# 학습 진행 : 인코딩 대상 적용
lencoder.fit(["paris", "paris", "tokyo", "amsterdam"])

# 모델 파라미터 확인
print(f"classes_ : {lencoder.classes_}, {lencoder.get_params()}")

# 변형 - 반환값(ndarray)
data = ["paris", "paris", "tokyo"]
lencoder.transform(data)
encoding = lencoder.transform(data)

# 복원/디코딩 => ndarray
lencoder.inverse_transform(encoding)

classes_ : ['amsterdam' 'paris' 'tokyo'], {}


array(['paris', 'paris', 'tokyo'], dtype='<U9')

[2] OrdinalEncoder <hr>
- 순서가 중요
- 성별일 경우 순서가 중요하지 않으므로 onHot 인코더 사용 권장
- 평점 같은 경우 순서가 중요함으로 Ordinal 인코더 사용 권장

In [179]:
# 관련 모듈 로딩
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

# 테스트용 데이터
data = pd.DataFrame({'gender':['F', 'F', 'M', 'F', 'M', 'UN']})

# 인스턴스 생성
ordEncoder = OrdinalEncoder()

# 데이터에 대한 인코딩 설정
ordEncoder.fit(data)

# 설정된 인코딩 정보 확인
print(f"feature_names_in_ : {ordEncoder.feature_names_in_}")
print(f"categories        : {ordEncoder.categories}")
print(f"n_features_in_    : {ordEncoder.n_features_in_}")

# 인코딩 진행
encoding = ordEncoder.transform(data)
print("인코딩 결과 : ", encoding)

# 복원/디코딩
print("복원/디코딩 결과 : ", ordEncoder.inverse_transform(encoding))

feature_names_in_ : ['gender']
categories        : auto
n_features_in_    : 1
인코딩 결과 :  [[0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [2.]]
복원/디코딩 결과 :  [['F']
 ['F']
 ['M']
 ['F']
 ['M']
 ['UN']]


[3] OneHotEncoder <hr>

In [180]:
# 관련 모듈 로딩
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# 테스트용 데이터
data = pd.DataFrame({'gender':['F', 'F', 'M', 'F', 'M', 'UN']})

# 인스턴스 생성 : 압축출력 설정 sparse_output : [기본값=True]
ohEncoder = OneHotEncoder(sparse_output=False)

# 데이터에 대한 인코딩 설정
ohEncoder.fit(data)

# 설정된 인코딩 정보 확인
print(f"categories        : {ohEncoder.categories}")
print(f"feature_names_in_ : {ohEncoder.feature_names_in_}")

# 인코딩 진행
encoding = ordEncoder.transform(data)
print("인코딩 결과 : ", encoding)

# 복원/디코딩
print("복원/디코딩 결과 : ", ordEncoder.inverse_transform(encoding))

categories        : auto
feature_names_in_ : ['gender']
인코딩 결과 :  [[0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [2.]]
복원/디코딩 결과 :  [['F']
 ['F']
 ['M']
 ['F']
 ['M']
 ['UN']]


[4] TargetEncoder <hr>

In [181]:
# 관련 모듈 로딩
from sklearn.preprocessing import TargetEncoder, OneHotEncoder

# 테스트 데이터
df = pd.DataFrame({
    "city": ["Seoul", "Busan", "Seoul", "Daegu", "Busan", "Seoul", "Incheon", "Daegu", "Busan", "Seoul"],
    "age":  [25, 32, 41, 23, 37, 29, 33, 45, 28, 40],
    "y":    [1, 0, 1, 0, 0, 1, 1, 0, 0, 1]  })

In [182]:
# 피쳐 컬럼 : city, age / 타겟 컬럼 : y
# 피쳐 컬럼 중 텍스트 데이터의 city ==> 수치화 즉, 인코딩

# OneHOT
# 인스턴스 생성 : 압축출력 설정 sparse_output : [기본값=True]
ohEncoder = OneHotEncoder(sparse_output=False, dtype=int)
cityArr = ohEncoder.fit_transform(df[['city']])
cityArr.tolist()

# df['city_onehot'] = cityArr.tolist()
# print(df)
cityArr = ohEncoder.fit_transform(df[['city']])
ageArr = df[df.columns[1:-1]].values
print(ageArr)

np.concatenate((cityArr, ageArr), axis=1)

[[25]
 [32]
 [41]
 [23]
 [37]
 [29]
 [33]
 [45]
 [28]
 [40]]


array([[ 0,  0,  0,  1, 25],
       [ 1,  0,  0,  0, 32],
       [ 0,  0,  0,  1, 41],
       [ 0,  1,  0,  0, 23],
       [ 1,  0,  0,  0, 37],
       [ 0,  0,  0,  1, 29],
       [ 0,  0,  1,  0, 33],
       [ 0,  1,  0,  0, 45],
       [ 1,  0,  0,  0, 28],
       [ 0,  0,  0,  1, 40]])